In [2]:
"""
Analyze MW power traces for each shot of experiment from R&S NRP18S power meter.
"""

import h5py
import os
import numpy as np

#from entire hdf5 data file, initialize run names we care about
absolute_path = os.path.abspath('')
rel_path = "MWPowerTraces_20240306.hdf" # in cell absorption data for push beam
filename = os.path.join(absolute_path, rel_path)
run_names = ['run_183233', 'run_183713'] #run names we care about (first two EOM on ,second two EOM off)

#input characterstic of power trace (duration of trace and number of points sampled). Comes from NRP18S.py file.
points = 200 #num of points in the trace measurement
duration = 100e-6 #in seconds


#open file and process data
with h5py.File(filename, 'r') as hdf:
    #dict1 is reduced dict of {run names : run data} only for runs we care about
    dict1 = {key: hdf[key] for key in run_names if key in hdf} 
    runs = [None for _ in range(len(run_names))]
    for key_index, key in enumerate(dict1):
        run = dict1[key]
        num_shots = len(run)
        sorted_keys = sorted(run.keys(), key=int) #sort run keys by true numerical order (not "string numerical" order")
        
        #run_shot is a list of the shots in each run in true numerical order. each shot is one mw power trace.
        run_shot = [None for _ in range(num_shots)]
        for key2 in sorted_keys:
            run_shot[int(key2)] = run[key2] 

        #extract data from runs (make list of lists of times and mw powers)
        layers, rows, cols = num_shots, 2, points
        run_data = [[[None for _ in range(cols)] for _ in range(rows)] for _ in range(layers)]

        for i in range(num_shots):
            times, mw_power = zip(*run_shot[i])
            run_data[i][0] = np.asarray(times)
            run_data[i][1] = np.asarray(mw_power)
        
        runs[key_index] = run_data 

    #dict2 is dict of {run names : runs} where runs is a list of 3D list containing all data for each shot (trace time and mw power)
    dict2 = {key: runs[run_names.index(key)] for key in run_names}

In [4]:
#go through dict2, analyze data, and extract power trace curves
#the val of dict2 (runs) is a 3D list in the form [shot number][time or power][value]
import matplotlib.pyplot as plt

rows, cols = len(run_names), 2
powers = [[None for _ in range(cols)] for _ in range(rows)]

#go through each run we care about
for key_index, key in enumerate(dict2):
    run = dict2[key]
    num_shots = len(run)

    timings = np.array([])
    sig = np.array([])
    
    #go through each shot in a run and assign the timing and power data accordingly
    for shot_index, shots in enumerate(run):
        times, mw_power = shots
        np.append(timings, times)
        np.append(sig, mw_power)

    powers[key_index][0] = timings
    powers[key_index][1] = sig
    
#dict3 is dict of {run names : power data} where power data is a 2D list in the form [][]
dict3 = {key: powers[run_names.index(key)] for key in run_names} 